# Human readable duration format

https://www.codewars.com/kata/52742f58faf5485cae000b9a/train/python

Your task in order to complete this Kata is to write a function which formats a duration, given as a number of seconds, in a human-friendly way.

The function must accept a non-negative integer. If it is zero, it just returns "now". Otherwise, the duration is expressed as a combination of years, days, hours, minutes and seconds.

It is much easier to understand with an example:

* For seconds = 62, your function should return 
    "1 minute and 2 seconds"
* For seconds = 3662, your function should return
    "1 hour, 1 minute and 2 seconds"
For the purpose of this Kata, a year is 365 days and a day is 24 hours.

Note that spaces are important.

Detailed rules

The resulting expression is made of components like 4 seconds, 1 year, etc. In general, a positive integer and one of the valid units of time, separated by a space. The unit of time is used in plural if the integer is greater than 1.

The components are separated by a comma and a space (", "). Except the last component, which is separated by " and ", just like it would be written in English.

A more significant units of time will occur before than a least significant one. Therefore, 1 second and 1 year is not correct, but 1 year and 1 second is.

Different components have different unit of times. So there is not repeated units like in 5 seconds and 1 second.

A component will not appear at all if its value happens to be zero. Hence, 1 minute and 0 seconds is not valid, but it should be just 1 minute.

A unit of time must be used "as much as possible". It means that the function should not return 61 seconds, but 1 minute and 1 second instead. Formally, the duration specified by of a component must not be greater than any valid more significant unit of time.

# Peter's breakdown and walkthrough

Let's take this complicated problem and break it down into smaller problems

> The function must accept a non-negative integer. If it is zero, it just returns "now"

In [5]:
def format_duration(seconds):
    if seconds == 0:
        return "now"

now_res = format_duration(0)
if now_res == "now":
    print("All good")
else:
    print(f"{now_res} is not now, try again")
    print 

All good


Moving on to
> Otherwise, the duration is expressed as a combination of years

Lets just make our function return the number of whole years that the seconds represents

In [19]:
seconds_in_a_year = 60 * 60 * 24 * 365
def format_duration(seconds):
    if seconds == 0:
        return "now"
    number_of_years = seconds // seconds_in_a_year
    return number_of_years

res_1 = format_duration(62)
if res_1 == 0:
    print("All good")
else:
    print(f"{res_1} is not 0 , try again")
    print 

res_2 = format_duration(31536000)
if res_2 == 1:
    print("All good")
else:
    print(f"{res_2} is not 1 , try again")
    print 

All good
All good


What about 
> ...  days, hours, minutes and seconds.

In [100]:
seconds_in_a_minute = 60
def minutes_in_seconds(seconds):
    return seconds // seconds_in_a_minute

seconds_in_an_hour = 60 * seconds_in_a_minute
def hours_in_seconds(seconds):
    return seconds // seconds_in_an_hour

seconds_in_a_day = 24 * seconds_in_an_hour
def days_in_seconds(seconds):
    return seconds // seconds_in_a_day

seconds_in_a_year = 365 * seconds_in_a_day
def years_in_seconds(seconds):
    return seconds // seconds_in_a_year



Combine the minutes and seconds functions together
try to produce a dict that results in this
```python
{
#    "years": 0,
#    "days": 0,
    "minutes": 1,
    "seconds": 2,
}
```

In [102]:
def format_duration(seconds):
    if seconds == 0:
        return "now"
    result_dict = {}
    remaining_seconds = seconds

    result_dict["year"] = years_in_seconds(remaining_seconds)
    remaining_seconds = remaining_seconds - result_dict["year"] * seconds_in_a_year
    result_dict["day"] = days_in_seconds(remaining_seconds)
    remaining_seconds = remaining_seconds - result_dict["day"] * seconds_in_a_day
    result_dict["hour"] = hours_in_seconds(remaining_seconds)
    remaining_seconds = remaining_seconds - result_dict["hour"] * seconds_in_an_hour
    result_dict["minute"] = minutes_in_seconds(remaining_seconds)
    remaining_seconds = remaining_seconds - result_dict["minute"] * seconds_in_a_minute
    result_dict["second"] = remaining_seconds
    return result_dict

expected = {
    "year": 0,
    "day": 0,
    "hour": 0,
    "minute": 1,
    "second": 2,
}

res_1 = format_duration(62)
if res_1 == expected:
    print(f"All good {res_1}")
else:
    print(f"{res_1} is not {expected} , try again")


All good {'year': 0, 'day': 0, 'hour': 0, 'minute': 1, 'second': 2}


In [103]:
print(format_duration( 2 * seconds_in_a_year + 3 * seconds_in_a_day + 7 * seconds_in_a_minute + 25 * 1))
print("2 years, 3 days, 7 minutes and 25 seconds")

{'year': 2, 'day': 3, 'hour': 0, 'minute': 7, 'second': 25}
2 years, 3 days, 7 minutes and 25 seconds


In [104]:
print(format_duration(2))
print("2 seconds")

{'year': 0, 'day': 0, 'hour': 0, 'minute': 0, 'second': 2}
2 seconds


Now we have a reliable function to ruten the counts for years, days, minutes and seconds we can start building our output.

Let's not go all the way to a full string, with the ", " and "and" separators, lets just try to produce a list like these
```python
["1 year", "2 days", "1 hour", "5 minutes", "1 second"]
["1 second"]
["2 years", "1 minute"]
```

In [105]:
def format_duration(seconds):
    if seconds == 0:
        return "now"
    result_dict = {}
    remaining_seconds = seconds

    result_dict["year"] = years_in_seconds(remaining_seconds)
    remaining_seconds = remaining_seconds - result_dict["year"] * seconds_in_a_year
    result_dict["day"] = days_in_seconds(remaining_seconds)
    remaining_seconds = remaining_seconds - result_dict["day"] * seconds_in_a_day
    result_dict["hour"] = hours_in_seconds(remaining_seconds)
    remaining_seconds = remaining_seconds - result_dict["hour"] * seconds_in_an_hour
    result_dict["minute"] = minutes_in_seconds(remaining_seconds)
    remaining_seconds = remaining_seconds - result_dict["minute"] * seconds_in_a_minute
    result_dict["second"] = remaining_seconds
    
    result_list = [
        f"{value} {key}{'s' if value > 1 else ''}" # the element in the list ( "1 second" or "5 minutes" )
        for key, value in result_dict.items()
        if value > 0
    ]
    return result_list

format_duration(5 * seconds_in_a_year + 4 * seconds_in_a_day + seconds_in_an_hour)

    

['5 years', '4 days', '1 hour']

Now we just need to combine them, adding a ", " between all of them except the last 2, and adding a " and " between that last two

In [108]:
", ".join(['5 years', '4 days', '1 hour'][0:-1]) + " and " 

'5 years, 4 days and '

In [118]:
lst = ['5 years', '4 days', '1 hour', '2 seconds']

return_str = ""
index = 0
for item in reversed(lst):
    index += 1
    if index == 1:
        return_str = item
    elif index == 2: # are we the second one?
        return_str = item + " and " + return_str
    else:
        return_str = item + ", " + return_str

print(return_str)

5 years, 4 days, 1 hour and 2 seconds
